# **PolypGen Challenge**

* Eleonora Catania Cecere
* Elisa Cevoli
* Tatiana Camila Cordoba Acosta
* Claudia Ottomano



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y torch torchvision torchaudio
!pip install -q torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -q mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 591.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q mmsegmentation==1.2.2
!pip install -q mmengine==0.10.3
!pip install -q ftfy==6.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.7 MB/s eta 0:00:00


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

2.1.0+cu121 True
1.2.2


In [5]:
# Librerie di base
import os
import re
import numpy as np

# Librerie per la manipolazione delle immagini
from PIL import Image
import cv2
import skimage

# Librerie scientifiche
from scipy import ndimage
from skimage.morphology import disk
from skimage.measure import label, regionprops
from scipy.ndimage import convolve, binary_opening, binary_fill_holes, binary_dilation

# Librerie Modello
from mmengine.config import Config
from mmengine.runner import Runner
from mmseg.datasets import BaseSegDataset
from mmseg.registry import DATASETS,RUNNERS
from mmseg.apis import init_model, inference_model

# Librerie per la visualizzazione
from tqdm import tqdm

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# DATASET

In [6]:
####################################
#### DO NOT CHANGE THIS CELL #######
####################################

# Classes and palette for VERSE dataset
classes = ('background', 'object')
palettePolypGen = [
    (0, 0, 0),       # background - black
    (255, 255, 255)  # object - white
]

@DATASETS.register_module()
class PolypGen(BaseSegDataset):
    METAINFO = dict(classes=classes, palette=palettePolypGen)

    def __init__(self, **kwargs):
        super().__init__(img_suffix='.jpg',
                         seg_map_suffix='_mask.png',
                         reduce_zero_label=False,
                         **kwargs)

In [7]:
group_name = 'Pepine'

PATHs

In [8]:
'''CARTELLA PRINCIPALE'''

working_folder = f'/content/drive/MyDrive/PolypGen/{group_name}'

checkpoint_path = f'{working_folder}/checkpoint.pth'
config_path = f'{working_folder}/config.py'

'''CARTELLA RISULTATI'''

results_folder  = f'{working_folder}/results_test'

# results_folder = '...'    # Inserire path per salvataggio risultati

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

'''CARICAMENTO SEQUENZE'''

test_img_folder = '/content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/PolypGen-EIM-24-25/sequenceData'

# test_img_folder = '...'    # Inserire path blind test sequence

In [9]:
''' CONFIGURAZIONE MODELLO '''

cfg = Config.fromfile(config_path)
cfg.device='cuda'

# Visualizzazione del config caricato
print(f'Config:\n{cfg.pretty_text}')

''' INIZIALIZZAZIONE MODELLO '''

# Initialize the model from the config and the checkpoint
model = init_model(cfg, checkpoint_path, 'cuda:0')

Config:
crop_size = (
    256,
    256,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        0,
        0,
        0,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        256,
        256,
    ),
    std=[
        1,
        1,
        1,
    ],
    type='SegDataPreProcessor')
data_root = '/content/drive/MyDrive/Lab_EIM/Progetto_finale_EIM/Divisione_dataset5'
dataset_type = 'PolypGen'
default_hooks = dict(
    checkpoint=dict(
        by_epoch=False,
        interval=448,
        max_keep_ckpts=1,
        rule='greater',
        save_best='mDice',
        type='CheckpointHook'),
    early_stopping=dict(
        min_delta=0.01,
        monitor='mIoU',
        patience=10,
        rule='greater',
        type='EarlyStoppingHook'),
    logger=dict(interval=50, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=d

/usr/local/lib/python3.11/dist-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/usr/local/lib/python3.11/dist-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/usr/local/lib/python3.11/dist-packages/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /content/drive/MyDrive/PolypGen/Pepine/checkpoint.pth


# Preprocessing TESTING

In [10]:
''' CROPPING '''

def crop_image(img_path, threshold_L=18, threshold_space=87, fill=False):

    # 1. Lettura dell'immagine e delle informazioni di base
    image = cv2.imread(img_path)
    file_name = os.path.basename(img_path)
    height, width = image.shape[:2] # dimensioni dell'immagine per upsampling

    # 2. Applicazione di un filtro gaussiano per ridurre il rumore
    image_gaus = cv2.GaussianBlur(image, (7, 7), sigmaX=5)

    # 3. Determinazione dello spazio colore (LAB o HSV) in base alla luminosità media dell'immagine
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean_value = np.mean(gray_img)

    if (mean_value >= threshold_space):
        image_lab = cv2.cvtColor(image_gaus, cv2.COLOR_BGR2LAB)
        image_space = image_lab
        num_iteration_dilation = 2
        num_iteration_erosion = 0
        val_sharp = 9
    else:
        image_hsv = cv2.cvtColor(image_gaus, cv2.COLOR_BGR2HSV)
        image_space = image_hsv
        num_iteration_dilation = 10
        num_iteration_erosion = 6
        val_sharp = 100
        fill = True

    # 4. Applicazione del filtro di sharpening per evidenziare i contorni
    kernel_sharpening =  np.array([[-1, -1, -1], [-1, val_sharp, -1], [-1, -1, -1]]) #15
    sharpened_image = cv2.filter2D(image_space, -1, kernel_sharpening)

    # 5. Creazione di una maschera binaria sulla base del primo canale dell'immagine elaborata
    maschera_binaria = (sharpened_image[...,0] > threshold_L).astype(np.uint8)

    # 6. Applicazione di operazioni morfologiche sulla maschera
    kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    maschera_opened = cv2.morphologyEx(maschera_binaria, cv2.MORPH_OPEN, kernel_open, iterations=2)
    maschera_opened = cv2.dilate(maschera_opened, (np.ones((5, 5), np.uint8)), iterations=num_iteration_dilation)

    # Riempimento dei buchi nelle maschere, solo nello spazio HSV
    if fill:
       maschera_opened = binary_fill_holes(maschera_opened).astype(np.uint8) #per sequenza 12

    # Erosione della maschera per affinare la segmentazione
    maschera_opened = cv2.erode(maschera_opened, (np.ones((5, 5), np.uint8)), iterations=num_iteration_erosion)

    # 7. Analisi della metà superiore dell'immagine per determinare il bounding box orizzontale
    height_half = int(image.shape[0] * 3 / 5) # Considera il 60% superiore dell'immagine
    maschera_upper_half = maschera_opened[:height_half, :]
    contours_upper_half, _ = cv2.findContours(maschera_upper_half, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Filtraggio dei contorni validi
    valid_contours_upper_half = [contour for contour in contours_upper_half if cv2.contourArea(contour) > 0]

    max_area_upper_half = 0
    largest_contour_upper_half = None

    # Identificazione del contorno più grande nella metà superiore
    for contour in valid_contours_upper_half:
        area = cv2.contourArea(contour)
        if area > max_area_upper_half:
            max_area_upper_half = area
            largest_contour_upper_half = contour

    # Rettangolo di delimitazione del contorno più grande nella porzione superiore
    if largest_contour_upper_half is not None:
        x_upper_half, y_upper_half, w_upper_half, h_upper_half = cv2.boundingRect(largest_contour_upper_half)
    else:
        x_upper_half, y_upper_half, w_upper_half, h_upper_half = 0, 0, image.shape[1], image.shape[0]  # Se non c'è contorno, usa l'intera immagine

    # 8. Analisi dell'intera immagine per determinare il bounding box verticale
    contours_full_image, _ = cv2.findContours(maschera_opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Filtraggio dei contorni validi
    valid_contours_full_image = [contour for contour in contours_full_image if cv2.contourArea(contour) > 0]

    max_area_full_image = 0
    largest_contour_full_image = None

    # Identificazione del contorno più grande nell'intera immagine
    for contour in valid_contours_full_image:
        area = cv2.contourArea(contour)
        if area > max_area_full_image:
            max_area_full_image = area
            largest_contour_full_image = contour

    # Rettangolo di delimitazione del contorno più grande nell'intera immagine
    if largest_contour_full_image is not None:
        x_full_image, y_full_image, w_full_image, h_full_image = cv2.boundingRect(largest_contour_full_image)
    else:
        x_full_image, y_full_image, w_full_image, h_full_image = 0, 0, image.shape[1], image.shape[0]  # Se non c'è contorno, usa l'intera immagine

    # 9. Creazione dell'immagine ritagliata utilizzando la larghezza calcolata nella metà superiore e l'altezza su tutta l'immagine
    cropped_image = image[y_full_image:y_full_image+h_full_image, x_upper_half:x_upper_half+w_upper_half]

    # 10. Controllo se il ritaglio è valido o se è troppo piccolo
    original_area = image.shape[0] * image.shape[1]
    cropped_area = cropped_image.shape[0] * cropped_image.shape[1]

    if cropped_area < original_area / 2:
        final_image = image  # Mantieni l'immagine originale
        x_crop, y_crop = 0, 0

    else:
        final_image = cropped_image  # Usa l'immagine ritagliata
        x_crop, y_crop = x_upper_half, y_full_image

    # Converti l'immagine nel formato RGB
    final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB)

    return final_image, width, height, x_crop, y_crop

''' RESIZE '''

def resize_mask(image_array, target_size, order=0, anti_aliasing=False):
    # Binarizzazione della maschera
    image_array = (image_array).astype(np.uint8)
    # Ridimensionamento della maschera (interpolazione nearest con order=0 e filtro anti_aliasing false)
    resized_img = skimage.transform.resize(image_array, target_size, anti_aliasing=anti_aliasing, order=order)
    return resized_img

def resize_image(image_array, target_size, order=1, anti_aliasing=True):
    height, width = image_array.shape[:2] # dimensioni originali per upsampling
    # Ridimensionamento dell'immagine (interpolazione bilineare con order=1 e filtro anti_aliasing)
    resized_img = skimage.transform.resize(image_array, target_size, anti_aliasing=anti_aliasing, order=order)
    # Conversione dell'immagine nel range [0, 255] e nel tipo uint8
    resized_img = (resized_img * 255).astype(np.uint8)
    return resized_img, width, height

''' RILEVAZIONE E RIMOZIONE DI SPECULAR HIGHLIGHTS '''

def reflex_detection(image_array):
    R = image_array[:, :, 0]  # Canale rosso
    G = image_array[:, :, 1]  # Canale verde
    # Identificazione pixel specular highlights
    mask = np.where((R > 220) & (G > 190), 255, 0).astype(np.uint8)
    return mask

def filter_small_and_large_regions(mask_array, max_area=500, large_area_threshold=1000):

    # Etichettatura delle componenti connesse
    labeled_mask, num_labels = ndimage.label(mask_array)

    # Inizializza le maschere per le regioni piccole e grandi
    mask_small = np.zeros_like(mask_array)
    mask_large = np.zeros_like(mask_array)

    # Iterazione su tutte le regioni identificate
    for label in range(1, num_labels + 1):
        region = (labeled_mask == label)
        area = np.sum(region)

        # Se l'area è piccola, maschera delle aree piccole
        if area < max_area:
            mask_small[region] = 255
        # Se l'area è grande ma inferiore ad una soglia, maschera delle aree grandi
        elif area >= max_area and area < large_area_threshold:
            mask_large[region] = 255

    return mask_small, mask_large

def dilate_mask(mask_array, kernel_size=5):
    # Kernel rettangolare per la dilatazione
    structure = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    dilated_mask = ndimage.binary_dilation(mask_array, structure=structure).astype(np.uint8) * 255
    return dilated_mask

def apply_opening(mask_array, kernel_size=5):
    # Kernel per l'operazione morfologica
    structure = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    processed_mask = binary_opening(mask_array, structure=structure).astype(np.uint8) * 255
    return Image.fromarray(processed_mask)

def calculate_texture_features(image, mask):
    # Calcolo delle caratteristiche della texture per i pixel riflettenti

    # 1. Conversione dell'immagine in scala di grigi
    gray_image = np.array(image.convert("L"))

    # 2. Definizione dei filtri di Sobel per il calcolo dei gradienti in 4 direzioni principali
    G_0 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]) # Direzione 0° (verticale)
    G_90 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]) # Direzione 90° (orizzontale)
    G_45 = np.array([[-2, -1, 0], [-1, 0, 1], [0, 1, 2]]) # Direzione 45° (diagonale: topLeft - downRight)
    G_135 = np.array([[0, 1, 2], [-1, 0, 1], [-2, -1, 0]]) # Direzione 135° (diagonale: topRight - downLeft)

    # 3. Applicazione dei filtri di Sobel per calcolare i gradienti in ogni direzione
    gradients = {
        0: convolve(gray_image, G_0, mode='nearest'),
        45: convolve(gray_image, G_45, mode='nearest'),
        90: convolve(gray_image, G_90, mode='nearest'),
        135: convolve(gray_image, G_135, mode='nearest'),
    }

    # 4. Calcolo del gradiente massimo tra tutte le direzioni
    max_gradient = np.maximum.reduce([gradients[0], gradients[45], gradients[90], gradients[135]])

    # 5. Creazione della matrice K per memorizzare la direzione del gradiente massimo
    K = np.zeros_like(max_gradient)

    # 6. Determinazione della direzione dominante per ogni pixel
    for angle, grad in gradients.items():
        K[max_gradient == grad] = angle

    # 7. Applicazione della maschera: i pixel non riflettenti sono impostati a 0
    K[mask == 0] = 0
    return K

def reflected_light_compensation(image_array, mask_array):
    # Compensazione delle aree riflettenti dell'immagine sostituendo i pixel riflettenti con pixel vicini non riflettenti

    # 1. Calcolo della direzione del gradiente per ciascun pixel riflettente
    K = calculate_texture_features(Image.fromarray(image_array), Image.fromarray(mask_array))

    compensated_image = image_array.copy()

    # 2. Identificazione dei pixel riflettenti (255 nella maschera)
    rows, cols = np.where(mask_array == 255)

    # 3. Per ogni pixel riflettente, cerca il primo pixel non riflettente nella direzione del gradiente K
    for x, y in zip(rows, cols):
        direction = K[x, y] # Ottieni la direzione del gradiente

        # 4. Determina la direzione di ricerca (dx, dy)
        dx, dy = 0, 0
        if direction == 0:  # Direzione orizzontale
            dx, dy = 0, 1
        elif direction == 45:  # Direzione 45 gradi
            dx, dy = -1, 1
        elif direction == 90:  # Direzione verticale
            dx, dy = -1, 0
        elif direction == 135:  # Direzione 135 gradi
            dx, dy = -1, -1

        # 5. Cerca il primo pixel non riflettente lungo la direzione calcolata
        nx, ny = x + dx, y + dy
                             # heigh                           # width
        while 0 <= nx < image_array.shape[0] and 0 <= ny < image_array.shape[1]:
            pixel_value = compensated_image[nx, ny] # Ottieni il valore RGB del pixel

            # Determina se il pixel è nero (soglia RGB < 50 per ogni canale)
            is_black = all(channel < 50 for channel in pixel_value)

            # Se il pixel non è riflettente e non è nero, usalo per sostituire il riflesso
            if mask_array[nx, ny] == 0 and not is_black:
                compensated_image[x, y] = compensated_image[nx, ny] # Sostituisci il pixel riflettente
                break

            # Continua nella stessa direzione
            nx, ny = nx + dx, ny + dy

    return compensated_image


def rgb_mask_inpainting(image_array, mask_array, patch_size=75):

    compensated_image = image_array.copy()

    # 1. Determina la metà del patch per definire l'area di ricerca
    half_patch = patch_size // 2  # Metà dimensione del patch

    # 2. Trova i pixel da ricostruire in base alla maschera (su tutti i canali)
    rows, cols, channels = np.where(mask_array > 0)

    # 3. Itera su tutti i pixel da ricostruire
    for x, y, channel in zip(rows, cols, channels):
        # 4. Definisce i limiti dell'intorno (evita di uscire dai bordi dell'immagine)
        x_min = max(x - half_patch, 0)
        x_max = min(x + half_patch + 1, image_array.shape[0])
        y_min = max(y - half_patch, 0)
        y_max = min(y + half_patch + 1, image_array.shape[1])

        # 5. Estrai la regione di interesse (patch) dall'immagine e dalla maschera
        patch = compensated_image[x_min:x_max, y_min:y_max]
        patch_mask = mask_array[x_min:x_max, y_min:y_max, channel]

        # 6. Filtra i pixel validi:
        # - Devono appartenere alla patch
        # - Non devono essere riflettenti (patch_mask == 0)
        # - Non devono essere troppo scuri (tutti i valori RGB > 25)
        valid_pixels = patch[(patch_mask == 0) & (np.all(patch > 25, axis=-1))]

        if len(valid_pixels) > 0:
            # 7. Se esistono pixel validi, sostituisce il valore riflettente con la media della regione
            compensated_image[x, y, channel] = np.mean(valid_pixels[:, channel], axis=0).astype(np.uint8)
        else:
            # 8. Se non ci sono pixel validi, mantiene il valore originale
            compensated_image[x, y, channel] = image_array[x, y, channel]

    return compensated_image

def remove_specular_highlights(original_image_array):

    # Step 1: Rilevazione degli specular highlights
    mask = reflex_detection(original_image_array)

    # Step 2: Filtraggio delle regioni piccole e grandi
    mask_small, mask_large = filter_small_and_large_regions(mask)

    # Step 3: Dilatazione della maschera small e apertura della maschera large
    refined_mask_small = dilate_mask(mask_small)
    refined_mask_large = apply_opening(mask_large)

    # Combinazione delle maschere
    refined_mask = refined_mask_small | refined_mask_large

    # Step 4: Ricostruzione dell'immagine sostituendo i riflessi con pixel adiacenti
    reconstructed_img = reflected_light_compensation(original_image_array, refined_mask)

    # Step 5: Applicazione di un filtro bilaterale SOLO alle aree della maschera
    filtered_masked_areas = cv2.bilateralFilter(
        cv2.bitwise_and(reconstructed_img, reconstructed_img, mask=dilate_mask(refined_mask)),
        d=9, sigmaColor=75, sigmaSpace=75
    )

    # Sostituisce le zone filtrate con interpolazione locale (inpainting basato su patch)
    image_with_selective_filtering = rgb_mask_inpainting(reconstructed_img, np.array(filtered_masked_areas), patch_size=15)

    # Step 6: Applicazione di un filtro mediano all'intera immagine per ridurre il rumore
    median_image_with_selective_filtering = cv2.medianBlur(image_with_selective_filtering, 5)

    # Step 7: Sharpening per ripristinare i dettagli dell'immagine dopo la filtratura
    kernel_sharpening = np.array([[-0.5, -0.5, -0.5], [-0.5, 5, -0.5], [-0.5, -0.5, -0.5]])
    sharpened_image = cv2.filter2D(median_image_with_selective_filtering, -1, kernel_sharpening)

    return sharpened_image

''' GAMMA CORRECTION '''

def gamma_correction(image_array, gamma = 1.2):
    gamma_corrected = np.zeros_like(image_array)
    look_up_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in range(256)], dtype=np.uint8)
    return cv2.LUT(image_array, look_up_table)

In [11]:
def preprocessing(img_path):

  ''' CROPPING '''
  img_cropped, width_original, height_original, x_crop, y_crop = crop_image(img_path)

  ''' RESIZE '''
  img_resize, width_pre_res, height_pre_res = resize_image(img_cropped, (256,256))

  ''' SPECULAR HIGHLIGHTS REMOVE '''
  img_specH = remove_specular_highlights(img_resize)

  ''' GAMMA CORRECTION '''
  img_gamma_corrected = gamma_correction(img_specH)

  output_image = Image.fromarray(img_gamma_corrected.astype(np.uint8))

  return output_image, width_original, height_original, x_crop, y_crop, width_pre_res, height_pre_res

# Postprocessing

In [12]:
''' PADDING per upsampling '''

def padding(image_array, width_target, height_target, x_crop, y_crop):

    # 1. Ottieni le dimensioni dell'immagine di input
    height_res, width_res = image_array.shape[:2]

    # 2. Calcola il padding necessario per raggiungere le dimensioni target
    padding_width = max(0, width_target - width_res)
    padding_height = max(0, height_target - height_res)

    # 3. Determina il padding da applicare su ciascun lato
    top = y_crop
    bottom = padding_height - top
    left = x_crop
    right = padding_width - left

    # 4. Applica il padding con bordi neri (valore 0)
    padded_image = cv2.copyMakeBorder(image_array, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

    # 5. Verifica che le dimensioni finali siano corrette
    assert padded_image.shape[:2] == (height_target, width_target), "Padding failed to match target size!"

    return padded_image

''' FILL HOLES '''

def apply_binary_fill_holes(mask):
    # Applica il fill sui buchi nella maschera binaria
    filled_mask = binary_fill_holes(mask).astype(np.uint8)
    return filled_mask


''' FILTRAGGIO AREE PICCOLE '''

def filter_small_areas(image, min=2500):
    # Etichetta le regioni connesse
    labeled_image = label(image)
    # Creazione di una nuova maschera filtrata
    filtered_mask = np.zeros_like(image)
    # Itera sulle regioni per filtrare quelle piccole
    for region in regionprops(labeled_image):
        if region.area >= min:
            filtered_mask[labeled_image == region.label] = 1
    return filtered_mask

''' DILATAZIONE '''

def apply_binary_dilation(mask, structure, iterations):
    # Applica dilatazione sulla maschera binaria
    dilated_mask = binary_dilation(mask, structure=structure, iterations=iterations).astype(np.uint8)
    return dilated_mask

In [13]:
def postprocessing(mask, width_original, height_original, x_crop, y_crop, width_pre_res, height_pre_res):

    ''' FILL HOLES '''
    filled_mask = apply_binary_fill_holes(mask)

    ''' FILTRAGGIO AREE PICCOLE '''
    refined_mask = filter_small_areas(filled_mask, min=350)

    ''' RESIZE (upsampling) '''
    mask_resized = resize_mask(np.array(refined_mask), (height_pre_res, width_pre_res), order=0, anti_aliasing=False)
    assert mask_resized.shape == (height_pre_res, width_pre_res), "Resize mismatch!"

    ''' PADDING (upsampling) '''
    mask_padded = padding(mask_resized, width_original, height_original, x_crop, y_crop)
    assert mask_padded.shape == (height_original, width_original), "Padding mismatch!"

    # binarizzazione
    mask_padded = mask_padded > 0

    ''' DILATAZIONE '''
    dilated_mask = apply_binary_dilation(mask_padded,  structure=disk(3), iterations=2)

    return dilated_mask

Organizzazione sequenze per inferenza

In [14]:
def build_sequence_dict(base_path, results_path):

    sequence_dict = {}

    # Ottieni le cartelle "seqX" ordinate correttamente per numero
    folders = [folder for folder in os.listdir(base_path) if folder.startswith("seq") and os.path.isdir(os.path.join(base_path, folder))]
    folders.sort(key=lambda x: int(x[3:]))  # Ordina basandosi sul numero dopo "seq"

    # Funzione per estrarre numeri dal nome del file
    def extract_numbers_from_filename(filename):
        return [int(num) for num in re.findall(r'\d+', filename)]

    # Itera su ogni cartella ordinata
    for folder_name in folders:
        folder_path = os.path.join(base_path, folder_name)

        # Nome della sequenza, ad esempio "seq1"
        sequence_name = folder_name

        # Path della sottocartella "images_seqX"
        images_folder = os.path.join(folder_path, f"images_{folder_name}")
        masks_folder = os.path.join(results_path, f"masks_pred_{folder_name}")

        if os.path.isdir(images_folder):
            # Lista delle immagini nella sottocartella
            image_files = sorted([f for f in os.listdir(images_folder) if f.endswith(".jpg")], key=extract_numbers_from_filename)

            # Aggiungi la lista di immagini e la destinazione al dizionario
            sequence_dict[sequence_name] = {
                "images": image_files,
                "destination": masks_folder
            }

    return sequence_dict

In [15]:
''' DIVIDI LE IMG SEQUENZE ORDINATE '''

sequence_dict = build_sequence_dict(test_img_folder, results_folder)

In [16]:
# Visualizza il dizionario con le immagini per ciascuna sequence
for sequence, details in sequence_dict.items():
    print(f"Sequence: {sequence}")
    print("Images:", details["images"])
    print("Dest:", details["destination"])
    print()

Sequence: seq1
Images: ['0_endocv2021_positive_69.jpg', '0_endocv2021_positive_77.jpg', '0_endocv2021_positive_78.jpg', '0_endocv2021_positive_84.jpg', '0_endocv2021_positive_95.jpg', '0_endocv2021_positive_104.jpg', '0_endocv2021_positive_106.jpg', '0_endocv2021_positive_141.jpg', '0_endocv2021_positive_143.jpg', '0_endocv2021_positive_166.jpg', '0_endocv2021_positive_167.jpg', '0_endocv2021_positive_169.jpg', '0_endocv2021_positive_172.jpg', '0_endocv2021_positive_178.jpg', '0_endocv2021_positive_190.jpg', '0_endocv2021_positive_199.jpg', '0_endocv2021_positive_206.jpg', '0_endocv2021_positive_208.jpg', '0_endocv2021_positive_221.jpg', '0_endocv2021_positive_224.jpg', '0_endocv2021_positive_233.jpg', '0_endocv2021_positive_248.jpg', '0_endocv2021_positive_250.jpg', '0_endocv2021_positive_272.jpg', '0_endocv2021_positive_280.jpg', '0_endocv2021_positive_284.jpg', '0_endocv2021_positive_293.jpg', '0_endocv2021_positive_299.jpg', '0_endocv2021_positive_300.jpg', '0_endocv2021_positive_3

In [17]:
# Loop attraverso le sequenze e le immagini corrispondenti
for sequence, details in sequence_dict.items():
    img_list = details["images"]
    destination_folder = details["destination"]
    sequence_number = int(sequence[3:])  # Estrarre il numero della sequenza

    # Dizionario per memorizzare i risultati per ciascuna immagine in una sequenza
    sequence_results = {}

    for img_name in tqdm(img_list, desc=f'Processing and evaluating sequence {sequence}'):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(test_img_folder, sequence, f'images_seq{sequence_number}',  img_name)
            if not os.path.exists(img_path):
                print(img_path)

            ###############################
            ##### 2D Pre-Processing #######
            ###############################

            img_prepocessed, width_original, height_original, x_crop, y_crop, width_pre_res, height_pre_res = preprocessing(img_path)

            # Save img to temporary file
            img_prepocessed.save('tmp_img.png')

            ###############################
            #######    Inference   ########
            ###############################

            result = inference_model(model, 'tmp_img.png')

            # Get data from the result
            pred_label = result.pred_sem_seg.data.squeeze()
            pred_label = pred_label.cpu().numpy().astype(np.uint8)

            ###############################
            ###### 2D Post-Processing #####
            ###############################

            pred_label_post = postprocessing(pred_label, width_original, height_original, x_crop, y_crop, width_pre_res, height_pre_res)

            ###############################
            #######   Salvataggio   #######
            ###############################

            # Salva la maschera predetta nella cartella di destinazione
            os.makedirs(destination_folder, exist_ok=True)
            pred_mask_path = os.path.join(destination_folder, img_name)
            Image.fromarray(pred_label_post.astype(np.uint8) * 255).save(pred_mask_path)

Processing and evaluating sequence seq23: 100%|██████████| 56/56 [01:06<00:00,  1.18s/it]


__________